In [1]:
import sys


no_of_cpu = 4
max_cores = 8
executor_mem = '26g'


Job_Name = 'Nandan : adhoc_req'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F


conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = ''
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
conf.set('spark.memory.fraction','.2')
conf.set('spark.driver.maxResultSize','1g')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
## changed from 4g to 2g
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
#conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#uncommented below
conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','200')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
16008,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-12-14 14:31:25.031278'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:

f=['CRITICAL ILLNESSPLAN',
'EMMA MASTER BOOKING PRODUCT',
'BTM MASTER PRODUCT',
'WORK SITE MARKETING',
'WORK SITE MARKETING - 18',
'WORK SITE MARKETING -33',
'WORK SITE MARKETING - BBPA',
'WELLSURANCE EXECUTIVE - IPA',
'WELLSURANCE FAMILY - IPA',
'WELLSURANCE WOMAN - IPA',
'RASHTRIYA SWASTHYA BIMA YOJANA',
'MEDIPRIME',
'MEDIPREM - DM',
'MEDISENIOR',
'MEDIRAKSHA',
'HOSPITALCARE',
'WELLSURANCE',
'WSM',
'MEDI-PLUS',
'WELLSURANCEEXECUTIVE',
'WELLSURANCEWOMAN',
'WELLSURANCEFAMILY',
'EXECUTIVEGUARD',
'WELLSURANCE SENIOR - IPA',
'WELLSURANCE EXECUTIVE-IPA MNTH',
'WELLSURANCE WOMAN-IPA MNTHLY',
'WELLSURANCE FAMILY - MNTHLY',
'WELLSURANCE EXE -IPA YRLY',
'WELLSURANCE WOMAN-IPA YRLY',
'WELLSURANCE FAMILY - YRLY',
'WELLSURANCE EXECUTIVE DM',
'WELLSURANCE WOMAN - DM',
'WELLSURANCE SENIOR - DM',
'WELLSURANCE FAMILY - DM',
'ACCIDENTSICKNESSHOSPITALCASH',
'PREMIER - MEDICARE',
'PROTECT - MEDICARE',
'MEDICARE',
'MEDICAREPLUS',
'AROGYASANJEEVANI',
'RETAILCOVIDPOLICY',
'CRITICARE',
'DIRECT MARKETING',
'MEDIPLUS',
'CRITIMEDICARE',
'FOUR WHEELER SHRIRAM',
'PRIVATE CAR INSURANCE',
'AUTOMOBILE',
'AUTO SHIELD',
'AUTOSECURE PRIVATE CAR',
'MOTOR',
'MOTOR INSURANCE',
'MOTORROADTRANSITPOLICY',
'AUTOSHIELD',
'PRIVATECARINSURANCEPOLICY',
'MOTORPRIVATECARPOLICYINSURANCE',
'SCOOTER/MOTOR CYCLE INSURANCE',
'SHRIRAM RSC SCOOTER EDI',
'TWO WHEELER SHRIRAM',
'AUTOSECURE TWO WHEELER',
'MOTORTWOWHEELERPOLICY',
'HOSPITAL CARE',
'MAHARAKSHA - PERSONAL INJURY',
'ACCIDENT GUARD - 5L',
'INJURY GUARD',
'FAMILYCARE PLAN',
'ACCIDENT GUARD - 5L OOC',
'SECURED FUTURE PLAN',
'ACCIDENT GUARD',
'CRITICARE PLAN -OTC',
'ACCIDENT SHIELD',
'SECUREDFUTUREPLAN',
'ACCIDENTGUARD',
'INJURYGUARD',
'ACCIDENTSHIELD',
'INDIVIDUAL PERSONAL ACCIDENT',
'FAMILYGUARD',
'INCOMEGUARD',
'ACCIDENT SHIELD BASE',
'HEALTH',
'HEALTH,PERSONAL ACCIDENT',
'PERSONAL ACCIDENT',
'SECUREINCOMELOW',
'SECUREINCOMEHIGH',
'FAMILYSECURE',
'ACCIDENTGUARDPLUS',
'IPA',
'MAHARAKSHA']


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#.filter(col('product_cd').isin('1601','1602','1603','1604''1606','3184','3122', '3121', '2888', '2896', '288801', '288802', '288803'))\

gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "registers",
                    "dbtable": "premium_register",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

premium_reg = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("customer_id", "product_name", 'minor_line_cd')\
                   .filter(col('product_name').isin(f))
premium_reg.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_id', 'product_name', 'minor_line_cd']

In [33]:
col_str=["golden_id", 'source_system_customer_id', 'active_flag']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"dbschema":"customermart",
"dbtable": "header",
"server.port": "1151-1160"}
header = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str).drop_duplicates().filter(col('active_flag').isin('1'))

header.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['golden_id', 'source_system_customer_id', 'active_flag']

In [34]:
df=premium_reg.join(header, premium_reg.customer_id==header.source_system_customer_id, 'inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
col_str=['golden_id','ckyc','kyc_id','kyc_datetime_stamp']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"dbtable": "customermart.customer_demographics",
"server.port": "1151-1160"}
cd = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
         .selectExpr(col_str).drop_duplicates()

cd.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['golden_id', 'ckyc', 'kyc_id', 'kyc_datetime_stamp']

In [36]:
from pyspark.sql.functions import coalesce,col,length,trim

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
cd=cd.select('golden_id',col('ckyc').cast('string'), col('kyc_id').cast('string'), col('kyc_datetime_stamp').cast('string') )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df=df.join(cd, df.golden_id==cd.golden_id).drop(cd.golden_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df=df.withColumn('is_kyc', when(coalesce(col('ckyc'),col('kyc_id'),col('kyc_datetime_stamp')).isNotNull(), lit(1)).otherwise(lit(0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[customer_id: string, product_name: string, minor_line_cd: string, golden_id: int, source_system_customer_id: string, active_flag: bigint, ckyc: string, kyc_id: string, kyc_datetime_stamp: string, is_kyc: int]

In [41]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

150036893

In [43]:
df.groupby(col('product_name'), col('is_kyc')).agg(countDistinct('golden_id')) \
    .show(100000, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+------+-------------------------+
|product_name                  |is_kyc|count(DISTINCT golden_id)|
+------------------------------+------+-------------------------+
|WELLSURANCE WOMAN - IPA       |0     |1                        |
|PRIVATECARINSURANCEPOLICY     |1     |384699                   |
|FAMILYSECURE                  |0     |1                        |
|ACCIDENTSICKNESSHOSPITALCASH  |1     |1289                     |
|WORK SITE MARKETING - 18      |0     |3                        |
|EMMA MASTER BOOKING PRODUCT   |0     |5                        |
|ACCIDENTGUARD                 |1     |6102                     |
|MEDIPLUS                      |1     |449                      |
|AROGYASANJEEVANI              |0     |2211                     |
|SECUREDFUTUREPLAN             |0     |63730                    |
|ACCIDENT GUARD - 5L           |1     |8                        |
|MEDIRAKSHA                    |1     |100                      |
|ACCIDENTG

In [44]:
df.groupby(col('product_name'), col('is_kyc')).agg(countDistinct('source_system_customer_id')) \
   .show(100000, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+------+-----------------------------------------+
|product_name                  |is_kyc|count(DISTINCT source_system_customer_id)|
+------------------------------+------+-----------------------------------------+
|WELLSURANCE WOMAN - IPA       |0     |1                                        |
|PRIVATECARINSURANCEPOLICY     |1     |414282                                   |
|FAMILYSECURE                  |0     |1                                        |
|ACCIDENTSICKNESSHOSPITALCASH  |1     |1505                                     |
|WORK SITE MARKETING - 18      |0     |3                                        |
|EMMA MASTER BOOKING PRODUCT   |0     |5                                        |
|ACCIDENTGUARD                 |1     |5891                                     |
|MEDIPLUS                      |1     |423                                      |
|AROGYASANJEEVANI              |0     |2254                                     |
|SECUREDFUTUREPL

In [ ]:
output_index = "cs_pro_p_26523"   ## change name
schema = "customermart"


import sys
try:

    cs_pro_p1.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [46]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…